# 2D Advection-Diffusion equation

in this notebook we provide a simple example of the DeepMoD algorithm and apply it on the 2D advection-diffusion equation. 

In [1]:
# General imports
import numpy as np
import torch
# DeepMoD functions
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library2D_third
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold,PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

# Settings for reproducibility
np.random.seed(1)
torch.manual_seed(1)

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Prepare the data

Next, we prepare the dataset.

In [12]:
data = loadmat('../Diffusion_2D_space81.mat')
data = np.real(data['Expression1']).reshape((81,81,81,4))

In [13]:
data.shape

(81, 81, 81, 4)

In [15]:
datax = data[:,:,:,1]
datay = data[:,:,:,2]
datat = data[:,:,:,0]
datau = data[:,:,:,3]

In [17]:
datau[:,:,1].shape

(81, 81)

In [67]:
number_of_samples = 10
tot_samples = number_of_samples*number_of_samples
Utrain = np.empty([tot_samples,data.shape[2]])
Xtrain = np.empty([tot_samples,data.shape[2],3])

In [68]:
Utrain.shape

(100, 81)

In [69]:
for i in np.arange(datax.shape[2]):
    idx = np.random.permutation(number_of_samples)
    idy = np.random.permutation(number_of_samples)
    Utrain[:,i] = np.array([datau[idx,k,i] for k in idy]).flatten()
    Xtrain[:,i,1] = np.array([datax[idx,k,i] for k in idy]).flatten()
    Xtrain[:,i,2] = np.array([datay[idx,k,i] for k in idy]).flatten()
    Xtrain[:,i,0] = np.array([datat[idx,k,i] for k in idy]).flatten()

In [70]:
Utrain.shape

(100, 81)

In [71]:
y = Utrain.flatten()
X = Xtrain.reshape(-1,3)

In [72]:


# Add noise 
noise_level = 0.0
y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)

# Randomize data 

idx = np.random.permutation(y.shape[0])
X_train = torch.tensor(X[idx, :], dtype=torch.float32, requires_grad=True).to(device)
y_train = torch.tensor(y_noisy[idx, :], dtype=torch.float32).to(device)


In [73]:

# Configure DeepMoD

network = NN(3, [40, 40, 40, 40], 1)
library = Library2D_third(poly_order=0) 
estimator = Threshold(0.05) 
sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
constraint = LeastSquares() 
model = DeepMoD(network, library, estimator, constraint).to(device)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) 
logdir='runs/test/'
train(model, X_train, y_train, optimizer,sparsity_scheduler, log_dir=logdir, split=0.8, max_iterations=50000, delta=1e-6, patience=200) 


 27550  MSE: 2.45e-02  Reg: 4.36e-10  L1: 2.32e+00 

KeyboardInterrupt: 

In [4]:
for i in time_range:
    
    # Downsample data and prepare data without noise:
    down_data= np.take(np.take(np.take(data,np.arange(0,x_dim,5),axis=0),np.arange(0,y_dim,5),axis=1),np.arange(0,t_dim,i),axis=2)
    print("Dowmsampled shape:",down_data.shape, "Total number of data points:", np.product(down_data.shape))
    index = len(np.arange(0,t_dim,i))    
    width, width_2, steps = down_data.shape
    x_arr, y_arr, t_arr = np.linspace(0,1,width), np.linspace(0,1,width_2), np.linspace(0,1,steps)
    x_grid, y_grid, t_grid = np.meshgrid(x_arr, y_arr, t_arr, indexing='ij')
    X, y = np.transpose((t_grid.flatten(), x_grid.flatten(), y_grid.flatten())), np.float32(down_data.reshape((down_data.size, 1)))
    
    
    # Add noise 
    noise_level = 0.0
    y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)

    # Randomize data 

    idx = np.random.permutation(y.shape[0])
    X_train = torch.tensor(X[idx, :], dtype=torch.float32, requires_grad=True).to(device)
    y_train = torch.tensor(y_noisy[idx, :], dtype=torch.float32).to(device)

    # Configure DeepMoD

    network = NN(3, [40, 40, 40, 40], 1)
    library = Library2D_third(poly_order=0) 
    estimator = Threshold(0.05) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) 
    logdir='final_runs/no_noise_x17/'+str(index)+'/'
    train(model, X_train, y_train, optimizer,sparsity_scheduler, log_dir=logdir, split=0.8, max_iterations=50000, delta=1e-6, patience=200) 


Dowmsampled shape: (17, 17, 41) Total number of data points: 11849
 49975  MSE: 8.70e-06  Reg: 8.07e-06  L1: 1.64e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (17, 17, 21) Total number of data points: 6069
 49975  MSE: 4.26e-06  Reg: 5.59e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (17, 17, 11) Total number of data points: 3179
 49975  MSE: 2.80e-06  Reg: 3.69e-06  L1: 1.47e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (17, 17, 7) Total number of data points: 2023
 49975  MSE: 3.36e-06  Reg: 2.87e-06  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (17, 17, 6) Total number of data points: 1734
  4700  MSE: 2.09e-04  Reg: 7.78e-06  L1: 1.00e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (17, 17, 5) Total number of data points: 1445
 49975  MSE: 4.50e-05  Reg: 1.17e-05  L1: 1.71e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (17, 17, 4) Total